In [1]:
import os
import numpy as np
import json
from PIL import Image
import requests
import cv2
from tqdm import tqdm
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.utils import shuffle
from keras.preprocessing import image as img
from keras.models import load_model

Using TensorFlow backend.


In [2]:
os.chdir('C:/Users/hp/Desktop/TCS-HumanAI-Submission-AGE-Ethnicity-Gender-Predictions')

In [3]:
with open('Face_Recognition.json', 'r') as infile:
    # Variable for building our JSON block
    json_block = []

    for line in infile:
        # Add the line to our JSON block
        json_block.append(line)
        # Check whether we closed our JSON block
        if line.startswith('}'):

            # Do something with the JSON dictionary
            json_dict = json.loads(''.join(json_block))
            print(json_dict)

            # Start a new block
            json_block = []

In [4]:
len(json_block)

120

In [10]:
for i in tqdm(range(121)):
    try:
        js = json.loads(json_block[i].replace("'", "\""))
        image = Image.open(requests.get(js['content'], stream=True).raw)
        image = np.asarray(image)
        gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        for d in js['annotation']:
            width = d['imageWidth']
            height = d['imageHeight']
            x1 = int(width*d['points'][0]['x'])
            y1 = int(height*d['points'][0]['y'])
            x2 = int(width*d['points'][1]['x'])
            y2 = int(height*d['points'][1]['y'])
            roi = gray[y1:y2,x1:x2]
            s = d['label'][0]
            for k in range(1,len(d['label'])):
                s = s+" "+ d['label'][k]
            cv2.imwrite(s+'.jpg',roi)
    except:
        continue

  6%|████▋                                                                             | 7/121 [01:06<21:56, 11.55s/it]C:\Users\hp\Anaconda3\lib\site-packages\PIL\Image.py:2655: UserWarning: image file could not be identified because WEBP support not installed
  warnings.warn(message)
100%|████████████████████████████████████████████████████████████████████████████████| 121/121 [20:25<00:00, 10.13s/it]


In [11]:
import shutil

In [13]:
for filename in os.listdir('./'):
    if filename.endswith('.jpg'):
        if 'below20' in filename:
            shutil.copy(filename,'./dataset/age/below_20/')
        if '20_30' in filename:
            shutil.copy(filename,'./dataset/age/20-30/')
        if '30_40' in filename:
            shutil.copy(filename,'./dataset/age/30-40/')
        if '40_50' in filename:
            shutil.copy(filename,'./dataset/age/40-50/')
        if 'above_50' in filename:
            shutil.copy(filename,'./dataset/age/above_50/')
        if 'Angry' in filename:
            shutil.copy(filename,'./dataset/emotions/angry/')
        if 'Happy' in filename:
            shutil.copy(filename,'./dataset/emotions/happy/')
        if 'Neutral' in filename:
            shutil.copy(filename,'./dataset/emotions/neutral/')
        if 'Sad' in filename:
            shutil.copy(filename,'./dataset/emotions/sad/')
        if 'Arab' in filename:
            shutil.copy(filename,'./dataset/ethnicity/arab/')
        if 'Asian' in filename:
            shutil.copy(filename,'./dataset/ethnicity/asian/')
        if 'Black' in filename:
            shutil.copy(filename,'./dataset/ethnicity/black/')
        if 'Hispanic' in filename:
            shutil.copy(filename,'./dataset/ethnicity/hispanic/')
        if 'Indian' in filename:
            shutil.copy(filename,'./dataset/ethnicity/indian/')
        if 'White' in filename:
            shutil.copy(filename,'./dataset/ethnicity/white/')
        if 'Female' in filename:
            shutil.copy(filename,'./dataset/gender/F/')
        if 'Male' in filename:
            shutil.copy(filename,'./dataset/gender/M/')

In [14]:
age_train_x = []
age_train_y = []

age_dict={
    'below_20':0,
     '20-30':1,
    '30-40':2,
    '40-50':3,
    'above_50':4
}

In [15]:
for item in os.listdir('./dataset/age'):
    for p in os.listdir('./dataset/age/'+str(item)):
        try:
            image = keras.preprocessing.image.load_img('./dataset/age/'+str(item)+"/"+str(p),target_size=(70,70))
            image = keras.preprocessing.image.img_to_array(image)
            image = image/255
            age_train_x.append(image)
            age_train_y.append(age_dict[item])
        except:
            continue

In [17]:
age_train_x = np.asarray(age_train_x)
age_train_y = np.asarray(age_train_y)

In [18]:
age_train_x.shape , age_train_y.shape

((76, 70, 70, 3), (76,))

In [19]:
age_train_y = keras.utils.to_categorical(age_train_y)

In [20]:
age_train_x, age_train_y = shuffle(age_train_x,age_train_y)

In [21]:
age_model = Sequential()

#Convolution layer 1
age_model.add(Conv2D(32,3,3,input_shape=(70,70,3),activation='relu'))
age_model.add(MaxPooling2D(pool_size=(2,2)))
#Convolution layer 2
age_model.add(Conv2D(64,3,3,activation='relu'))
age_model.add(MaxPooling2D(pool_size=(2,2)))
#Flattening 
age_model.add(Flatten())
#Feeding into fully Connected layers
age_model.add(Dense(128,activation='relu'))
age_model.add(Dense(64,activation='relu'))
#output layer
age_model.add(Dense(age_train_y.shape[1],activation='softmax')) 
age_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

W0824 20:39:29.343574 10876 deprecation_wrapper.py:119] From C:\Users\hp\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(70, 70, 3..., activation="relu")`
  after removing the cwd from sys.path.
W0824 20:39:29.380475 10876 deprecation_wrapper.py:119] From C:\Users\hp\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0824 20:39:29.385429 10876 deprecation_wrapper.py:119] From C:\Users\hp\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0824 20:39:29.413355 10876 deprecation_wrapper.py:119] From C:\Users\hp\Anacond

In [22]:
age_model.fit(age_train_x,age_train_y,epochs=100,batch_size=32,validation_split=0.1)

W0824 20:39:39.382693 10876 deprecation.py:323] From C:\Users\hp\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0824 20:39:39.448516 10876 deprecation_wrapper.py:119] From C:\Users\hp\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 68 samples, validate on 8 samples
Epoch 1/100
68/68 [==============================] - ETA: 20s - loss: 1.6126 - acc: 0.15 - ETA: 1s - loss: 1.6597 - acc: 0.1719 - 18s 269ms/step - loss: 1.6769 - acc: 0.1618 - val_loss: 1.6049 - val_acc: 0.0000e+00
Epoch 2/100
68/68 [==============================] - ETA: 0s - loss: 1.5578 - acc: 0.281 - ETA: 0s - loss: 1.5805 - acc: 0.281 - 0s 2ms/step - loss: 1.5792 - acc: 0.2941 - val_loss: 1.6848 - val_acc: 0.0000e+00
Epoch 3/100
68/68 [==============================] - ETA: 0s - loss: 1.5658 - acc: 0.281 - ETA: 0s - loss: 1.5495 - acc: 0.281 - 0s 2ms/step - loss: 1.5579 - acc: 0.2794 - val_loss: 1.6198 - val_acc: 0.0000e+00
Epoch 4/100
68/68 [==============================] - ETA: 0s - loss: 1.5112 - acc: 0.281 - ETA: 0s - loss: 1.5110 - acc: 0.343 - 0s 2ms/step - loss: 1.5205 - acc: 0.3529 - val_loss: 1.5571 - val_acc: 0.5000
Epoch 5/100
68/68 [==============================] - ETA: 0s - loss: 1.4346 - acc: 0.437 - ETA: 0s - loss: 1.4835

68/68 [==============================] - ETA: 0s - loss: 0.0085 - acc: 1.000 - ETA: 0s - loss: 0.0423 - acc: 0.968 - 0s 2ms/step - loss: 0.0546 - acc: 0.9559 - val_loss: 5.4779 - val_acc: 0.1250
Epoch 80/100
68/68 [==============================] - ETA: 0s - loss: 0.1177 - acc: 0.937 - ETA: 0s - loss: 0.0598 - acc: 0.968 - 0s 2ms/step - loss: 0.0566 - acc: 0.9706 - val_loss: 5.2132 - val_acc: 0.1250
Epoch 81/100
68/68 [==============================] - ETA: 0s - loss: 0.0703 - acc: 0.968 - ETA: 0s - loss: 0.0789 - acc: 0.968 - 0s 2ms/step - loss: 0.0743 - acc: 0.9706 - val_loss: 5.1204 - val_acc: 0.1250
Epoch 82/100
68/68 [==============================] - ETA: 0s - loss: 0.0113 - acc: 1.000 - ETA: 0s - loss: 0.0870 - acc: 0.968 - 0s 2ms/step - loss: 0.0819 - acc: 0.9706 - val_loss: 5.1086 - val_acc: 0.1250
Epoch 83/100
68/68 [==============================] - ETA: 0s - loss: 0.0830 - acc: 0.968 - ETA: 0s - loss: 0.0801 - acc: 0.968 - 0s 2ms/step - loss: 0.0755 - acc: 0.9706 - val_loss

In [23]:
age_model.save('age_model.h5')

In [24]:
gender_train_x = []
gender_train_y=[]

gender_dict = {
    'M':0,
    'F':1
}

for item in os.listdir('./dataset/gender'):
    for p in os.listdir('./dataset/gender/'+str(item)):
        try:
            image = keras.preprocessing.image.load_img('./dataset/gender/'+str(item)+"/"+str(p),target_size=(70,70))
            image = keras.preprocessing.image.img_to_array(image)
            image = image/255
            gender_train_x.append(image)
            gender_train_y.append(gender_dict[item])
        except:
            continue

gender_train_x = np.asarray(gender_train_x)
gender_train_y = np.asarray(gender_train_y)

gender_train_x, gender_train_y = shuffle(gender_train_x,gender_train_y)

gender_train_y=keras.utils.to_categorical(gender_train_y)

In [25]:
gender_model = Sequential()

#Convolution layer 1
gender_model.add(Conv2D(32,3,3,input_shape=(70,70,3),activation='relu'))
gender_model.add(MaxPooling2D(pool_size=(2,2)))
#Convolution layer 2
gender_model.add(Conv2D(64,3,3,activation='relu'))
gender_model.add(MaxPooling2D(pool_size=(2,2)))
#Flattening 
gender_model.add(Flatten())
#Feeding into fully Connected layers
gender_model.add(Dense(128,activation='relu'))
gender_model.add(Dense(64,activation='relu'))
#output layer
gender_model.add(Dense(2,activation='softmax')) 
gender_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(70, 70, 3..., activation="relu")`
  after removing the cwd from sys.path.
C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  import sys


In [26]:
gender_model.fit(gender_train_x,gender_train_y,epochs=100,batch_size=32,validation_split=0.1)

Train on 64 samples, validate on 8 samples
Epoch 1/100
64/64 [==============================] - ETA: 0s - loss: 0.6940 - acc: 0.531 - 1s 14ms/step - loss: 0.6936 - acc: 0.5625 - val_loss: 0.5655 - val_acc: 0.7500
Epoch 2/100
64/64 [==============================] - ETA: 0s - loss: 1.1414 - acc: 0.406 - 0s 2ms/step - loss: 0.9139 - acc: 0.4375 - val_loss: 0.9741 - val_acc: 0.2500
Epoch 3/100
64/64 [==============================] - ETA: 0s - loss: 0.8052 - acc: 0.375 - 0s 2ms/step - loss: 0.7002 - acc: 0.5312 - val_loss: 0.8127 - val_acc: 0.2500
Epoch 4/100
64/64 [==============================] - ETA: 0s - loss: 0.6891 - acc: 0.437 - 0s 2ms/step - loss: 0.6594 - acc: 0.5312 - val_loss: 0.7557 - val_acc: 0.2500
Epoch 5/100
64/64 [==============================] - ETA: 0s - loss: 0.6222 - acc: 0.656 - 0s 2ms/step - loss: 0.6400 - acc: 0.5469 - val_loss: 0.7567 - val_acc: 0.3750
Epoch 6/100
64/64 [==============================] - ETA: 0s - loss: 0.5969 - acc: 0.687 - 0s 2ms/step - loss: 

64/64 [==============================] - ETA: 0s - loss: 3.1472e-05 - acc: 1.000 - 0s 2ms/step - loss: 2.9369e-05 - acc: 1.0000 - val_loss: 2.4409 - val_acc: 0.6250
Epoch 95/100
64/64 [==============================] - ETA: 0s - loss: 2.6181e-05 - acc: 1.000 - 0s 2ms/step - loss: 2.8460e-05 - acc: 1.0000 - val_loss: 2.4451 - val_acc: 0.6250
Epoch 96/100
64/64 [==============================] - ETA: 0s - loss: 2.2778e-05 - acc: 1.000 - 0s 2ms/step - loss: 2.7666e-05 - acc: 1.0000 - val_loss: 2.4507 - val_acc: 0.6250
Epoch 97/100
64/64 [==============================] - ETA: 0s - loss: 3.2517e-05 - acc: 1.000 - 0s 2ms/step - loss: 2.6853e-05 - acc: 1.0000 - val_loss: 2.4533 - val_acc: 0.6250
Epoch 98/100
64/64 [==============================] - ETA: 0s - loss: 2.0232e-05 - acc: 1.000 - 0s 2ms/step - loss: 2.6111e-05 - acc: 1.0000 - val_loss: 2.4567 - val_acc: 0.6250
Epoch 99/100
64/64 [==============================] - ETA: 0s - loss: 2.7443e-05 - acc: 1.000 - 0s 2ms/step - loss: 2.5328e

In [27]:
gender_model.save('gender_model.h5')

In [28]:
ethnicity_dict={
    'arab':0,
    'asian':1,
    'black':2,
    'hispanic':3,
    'indian':4,
    'white':5
}

ethnicity_train_x = []
ethnicity_train_y=[]

for item in os.listdir('./dataset/ethnicity'):
    for p in os.listdir('./dataset/ethnicity/'+str(item)):
        try:
            image = keras.preprocessing.image.load_img('./dataset/ethnicity/'+str(item)+"/"+str(p),target_size=(70,70))
            image = keras.preprocessing.image.img_to_array(image)
            image = image/255
            ethnicity_train_x.append(image)
            ethnicity_train_y.append(ethnicity_dict[item])
        except:
            continue
            
ethnicity_train_x = np.asarray(ethnicity_train_x)
ethnicity_train_y = np.asarray(ethnicity_train_y)

ethnicity_train_y=keras.utils.to_categorical(ethnicity_train_y)

ethnicity_train_x, ethnicity_train_y = shuffle(ethnicity_train_x,ethnicity_train_y)

In [29]:
ethnicity_model = Sequential()

#Convolution layer 1
ethnicity_model.add(Conv2D(32,3,3,input_shape=(70,70,3),activation='relu'))
ethnicity_model.add(MaxPooling2D(pool_size=(2,2)))
#Convolution layer 2
ethnicity_model.add(Conv2D(64,3,3,activation='relu'))
ethnicity_model.add(MaxPooling2D(pool_size=(2,2)))
#Flattening 
ethnicity_model.add(Flatten())
#Feeding into fully Connected layers
ethnicity_model.add(Dense(128,activation='relu'))
ethnicity_model.add(Dense(64,activation='relu'))
#output layer
ethnicity_model.add(Dense(ethnicity_train_y.shape[1],activation='softmax')) 
ethnicity_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(70, 70, 3..., activation="relu")`
  after removing the cwd from sys.path.
C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  import sys


In [30]:
ethnicity_model.fit(ethnicity_train_x,ethnicity_train_y,epochs=100,batch_size=32,validation_split=0.1)

Train on 64 samples, validate on 8 samples
Epoch 1/100
64/64 [==============================] - ETA: 0s - loss: 1.7839 - acc: 0.250 - 1s 15ms/step - loss: 1.6573 - acc: 0.2188 - val_loss: 1.9830 - val_acc: 0.3750
Epoch 2/100
64/64 [==============================] - ETA: 0s - loss: 1.5640 - acc: 0.500 - 0s 2ms/step - loss: 1.5193 - acc: 0.4219 - val_loss: 1.5557 - val_acc: 0.3750
Epoch 3/100
64/64 [==============================] - ETA: 0s - loss: 1.2933 - acc: 0.593 - 0s 2ms/step - loss: 1.3880 - acc: 0.5312 - val_loss: 1.5828 - val_acc: 0.2500
Epoch 4/100
64/64 [==============================] - ETA: 0s - loss: 1.2492 - acc: 0.593 - 0s 2ms/step - loss: 1.3078 - acc: 0.5469 - val_loss: 1.7127 - val_acc: 0.3750
Epoch 5/100
64/64 [==============================] - ETA: 0s - loss: 1.2869 - acc: 0.500 - 0s 2ms/step - loss: 1.1802 - acc: 0.5469 - val_loss: 1.7862 - val_acc: 0.3750
Epoch 6/100
64/64 [==============================] - ETA: 0s - loss: 1.0113 - acc: 0.593 - 0s 2ms/step - loss: 

Epoch 49/100
64/64 [==============================] - ETA: 0s - loss: 0.0274 - acc: 0.968 - 0s 2ms/step - loss: 0.0365 - acc: 0.9688 - val_loss: 4.9712 - val_acc: 0.2500
Epoch 50/100
64/64 [==============================] - ETA: 0s - loss: 0.0538 - acc: 0.968 - 0s 2ms/step - loss: 0.0337 - acc: 0.9844 - val_loss: 5.0082 - val_acc: 0.2500
Epoch 51/100
64/64 [==============================] - ETA: 0s - loss: 0.0157 - acc: 1.000 - 0s 2ms/step - loss: 0.0277 - acc: 0.9844 - val_loss: 5.0807 - val_acc: 0.2500
Epoch 52/100
64/64 [==============================] - ETA: 0s - loss: 0.0457 - acc: 0.968 - 0s 2ms/step - loss: 0.0242 - acc: 0.9844 - val_loss: 5.3068 - val_acc: 0.2500
Epoch 53/100
64/64 [==============================] - ETA: 0s - loss: 0.0015 - acc: 1.000 - 0s 2ms/step - loss: 0.0315 - acc: 0.9844 - val_loss: 5.4495 - val_acc: 0.2500
Epoch 54/100
64/64 [==============================] - ETA: 0s - loss: 0.0099 - acc: 1.000 - 0s 2ms/step - loss: 0.0290 - acc: 0.9844 - val_loss: 5.458

In [31]:
ethnicity_model.save('ethnicity_model.h5')

In [32]:
emotion_dict={
    'angry':0,
    'happy':1,
    'neutral':2,
    'sad':3
}
emotions_train_x = []
emotions_train_y=[]

for item in os.listdir('./dataset/emotions'):
    for p in os.listdir('./dataset/emotions/'+str(item)):
        try:
            image = keras.preprocessing.image.load_img('./dataset/emotions/'+str(item)+"/"+str(p),target_size=(70,70))
            image = keras.preprocessing.image.img_to_array(image)
            image = image/255
            emotions_train_x.append(image)
            emotions_train_y.append(emotion_dict[item])
        except:
            continue
emotions_train_x = np.asarray(emotions_train_x)
emotions_train_y = np.asarray(emotions_train_y)

emotions_train_y=keras.utils.to_categorical(emotions_train_y)

emotions_train_x, emotions_train_y= shuffle(emotions_train_x,emotions_train_y)

In [33]:
emotions_model = Sequential()

#Convolution layer 1
emotions_model.add(Conv2D(32,3,3,input_shape=(70,70,3),activation='relu'))
emotions_model.add(MaxPooling2D(pool_size=(2,2)))
#Convolution layer 2
emotions_model.add(Conv2D(64,3,3,activation='relu'))
emotions_model.add(MaxPooling2D(pool_size=(2,2)))
#Flattening 
emotions_model.add(Flatten())
#Feeding into fully Connected layers
emotions_model.add(Dense(128,activation='relu'))
emotions_model.add(Dense(64,activation='relu'))
#output layer
emotions_model.add(Dense(emotions_train_y.shape[1],activation='softmax')) 
emotions_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(70, 70, 3..., activation="relu")`
  after removing the cwd from sys.path.
C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  import sys


In [34]:
emotions_model.fit(emotions_train_x,emotions_train_y,epochs=100,batch_size=32,validation_split=0.1)

Train on 68 samples, validate on 8 samples
Epoch 1/100
68/68 [==============================] - ETA: 0s - loss: 1.3849 - acc: 0.125 - ETA: 0s - loss: 1.4999 - acc: 0.187 - 1s 15ms/step - loss: 1.4581 - acc: 0.2206 - val_loss: 1.0327 - val_acc: 0.7500
Epoch 2/100
68/68 [==============================] - ETA: 0s - loss: 2.0105 - acc: 0.375 - ETA: 0s - loss: 1.6178 - acc: 0.453 - 0s 2ms/step - loss: 1.5826 - acc: 0.4706 - val_loss: 1.1434 - val_acc: 0.7500
Epoch 3/100
68/68 [==============================] - ETA: 0s - loss: 1.2799 - acc: 0.468 - ETA: 0s - loss: 1.1943 - acc: 0.453 - 0s 2ms/step - loss: 1.1835 - acc: 0.4559 - val_loss: 1.2154 - val_acc: 0.1250
Epoch 4/100
68/68 [==============================] - ETA: 0s - loss: 1.2177 - acc: 0.312 - ETA: 0s - loss: 1.1468 - acc: 0.359 - 0s 2ms/step - loss: 1.1666 - acc: 0.3529 - val_loss: 1.1704 - val_acc: 0.2500
Epoch 5/100
68/68 [==============================] - ETA: 0s - loss: 1.3175 - acc: 0.218 - ETA: 0s - loss: 1.1371 - acc: 0.484 -

68/68 [==============================] - ETA: 0s - loss: 8.0423e-04 - acc: 1.000 - ETA: 0s - loss: 7.0228e-04 - acc: 1.000 - 0s 2ms/step - loss: 6.8410e-04 - acc: 1.0000 - val_loss: 2.8989 - val_acc: 0.7500
Epoch 79/100
68/68 [==============================] - ETA: 0s - loss: 9.3015e-04 - acc: 1.000 - ETA: 0s - loss: 6.8432e-04 - acc: 1.000 - 0s 2ms/step - loss: 6.6170e-04 - acc: 1.0000 - val_loss: 2.9121 - val_acc: 0.7500
Epoch 80/100
68/68 [==============================] - ETA: 0s - loss: 7.1406e-04 - acc: 1.000 - ETA: 0s - loss: 6.6046e-04 - acc: 1.000 - 0s 2ms/step - loss: 6.4247e-04 - acc: 1.0000 - val_loss: 2.9246 - val_acc: 0.7500
Epoch 81/100
68/68 [==============================] - ETA: 0s - loss: 4.8776e-04 - acc: 1.000 - ETA: 0s - loss: 6.4326e-04 - acc: 1.000 - 0s 2ms/step - loss: 6.2281e-04 - acc: 1.0000 - val_loss: 2.9324 - val_acc: 0.7500
Epoch 82/100
68/68 [==============================] - ETA: 0s - loss: 6.2067e-04 - acc: 1.000 - ETA: 0s - loss: 6.0428e-04 - acc: 1.0

In [35]:
emotions_model.save('emotions_model.h5')